# Query result content

Queries return a `Result` object that we can customize according to our needs.

In [1]:
%pip install superlinked==29.2.1

In [2]:
import pandas as pd
from superlinked import framework as sl

pd.set_option("display.max_colwidth", 100)

Let's create a simple config and ingest two entities of text just for demonstration purposes.

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String
    like_count: sl.Integer


paragraph = Paragraph()

body_space = sl.TextSimilaritySpace(text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2")
like_space = sl.NumberSpace(number=paragraph.like_count, min_value=0, max_value=100, mode=sl.Mode.MAXIMUM)
# indices can be built on top of multiple spaces as simple as that
paragraph_index = sl.Index([body_space, like_space], fields=paragraph.like_count)

source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

source.put(
    [
        {
            "id": "paragraph-1",
            "body": "Glorious animals live in the wilderness.",
            "like_count": 75,
        },
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
            "like_count": 10,
        },
    ]
)

query_base = (
    sl.Query(
        paragraph_index,
        weights={
            body_space: 1.0,
            like_space: 1.0,
        },
    )
    .find(paragraph)
    .similar(body_space, "What makes the AI industry go forward?")
    .filter(paragraph.like_count > 15)
    .include_metadata()  # this clause makes the result entries contain metadata, like partial scores
)

## General result structure

Result consist of entries

In [4]:
result = app.query(query_base)

result.entries

[ResultEntry(id='paragraph-1', fields={}, metadata=ResultEntryMetadata(score=0.4417109896633091, partial_scores=[-0.020228776592334182, 0.46193976625564326], vector_parts=[]))]

and entries have metadata, containing the score for example

In [5]:
result.entries[0].metadata.score

0.4417109896633091

and partial scores - a score breakdown per spaces in the index.

In [6]:
result.entries[0].metadata.partial_scores

[-0.020228776592334182, 0.46193976625564326]

these naturally add to up to the score itself...

In [7]:
sum(result.entries[0].metadata.partial_scores) == result.entries[0].metadata.score

True

...and refer to the spaces in the order those were supplied at index creation.

In [8]:
partial_scores_dict = dict(zip(["body_space", "like_space"], result.entries[0].metadata.partial_scores))
partial_scores_dict

{'body_space': -0.020228776592334182, 'like_space': 0.46193976625564326}

### Display results

A nice way to show results in a `pandas DataFrame` is the following:

In [9]:
sl.PandasConverter.to_pandas(result)

,id,similarity_score
0,paragraph-1,0.441711


and we can add the partial scores to the dataframe.

In [10]:
def get_partial_score_df(query_result: sl.QueryResult) -> pd.DataFrame:
    partial_score_df: pd.DataFrame = pd.DataFrame(
        [[entry.id] + list(entry.metadata.partial_scores) for entry in query_result.entries],
        columns=["id", "body_space", "category_space"],
    )
    return sl.PandasConverter.to_pandas(query_result).merge(partial_score_df, on="id")


get_partial_score_df(result)

,id,similarity_score,body_space,category_space
0,paragraph-1,0.441711,-0.020229,0.46194


### Result metadata

Result metadata contains several useful things, like the vector the KNN search was carried out with in the VDB,

In [11]:
result.metadata.search_vector[:5]

[0.018769700726501557,
 0.03672832746481938,
 -0.0372933507748233,
 -0.03523925553426185,
 -0.023609617164574218]

and contains the schema, of which the results are from

In [12]:
result.metadata.schema_name

'Paragraph'

also the actual inputs to the query are also accessible. Many of these parameters are implicit, hence the generated names. You can always make them explicit, thereby giving them names.


In [13]:
result.metadata.search_params

{'similar_filter_TextSimilaritySpace_e74d_Paragraph_body_value_param__': 'What makes the AI industry go forward?',
 'similar_filter_TextSimilaritySpace_e74d_Paragraph_body_weight_param__': 1.0,
 'hard_filter_like_count_be_greater_than_param__': 15,
 'limit_param__': -1,
 'radius_param__': None,
 'select_param__': [],
 'space_weight_TextSimilaritySpace_e74d_param__': 1.0,
 'space_weight_NumberSpace_b3ce_param__': 1.0}

Notice `'hard_filter_like_count_be_greater_than_param__': 15` describes the .filter clause's effect for example.

### Configure what fields are returned in the Result

#### .select_all()

This clause makes the query result return all the fields the entities have.

A query without any select clause will return none of the fields.

In [14]:
result_no_select_clause = app.query(query_base)
result_no_select_clause.entries[0].fields

{}

Notice all 2 fields (both `body` and `like_count` of `paragraph`) are returned with the `.select_all()` clause.

In [15]:
query_select_all = query_base.select_all()
result_select_all = app.query(query_select_all)
result_select_all.entries[0].fields

{'body': 'Glorious animals live in the wilderness.', 'like_count': 75}

#### .select()

The select clause enables fine-grained field selection.

In [16]:
query_fix_select = query_base.select(paragraph.body)
result_fix_select = app.query(query_fix_select)
result_fix_select.entries[0].fields

{'body': 'Glorious animals live in the wilderness.'}

It can also be used with an `sl.Param`, filled query time.

In [17]:
query_select = query_base.select(sl.Param("select_fields"))

Fields can be referenced via string names,

In [18]:
result_body = app.query(query_select, select_fields=["body"])
result_body.entries[0].fields

{'body': 'Glorious animals live in the wilderness.'}

or referring to the schema field itself.

In [19]:
result_like = app.query(query_select, select_fields=[paragraph.like_count])
result_like.entries[0].fields

{'like_count': 75}

An empty select clause will result in no fields are returned for efficiency (ids are always returned of course),

In [20]:
result_empty = app.query(query_select)
result_empty.entries[0].fields

{}

and referencing ways can even be mixed in the same clause.

In [21]:
result_both = app.query(query_select, select_fields=[paragraph.body, "like_count"])
result_both.entries[0].fields

{'body': 'Glorious animals live in the wilderness.', 'like_count': 75}